# 54-异步最佳实践

## 📚 用途说明

**学习目标**：
- 掌握异步编程的最佳实践和设计模式
- 学会异步系统的架构设计和部署策略
- 理解异步代码的测试和调试方法
- 能够构建生产级的异步应用系统

**前置要求**：
- 已完成异步性能优化学习
- 熟练掌握异步编程核心技术
- 了解系统架构和设计模式
- 理解生产环境部署要求

**与LangChain关联**：
- 最佳实践确保LangChain的稳定性
- 架构设计支持LangChain的扩展性
- 测试方法保障LangChain的质量
- 部署策略实现LangChain的可靠性

---

## 🔢 知识点覆盖

### 6.9 异步最佳实践 [⭐⭐进阶]
**知识点说明**：异步最佳实践是构建高质量异步系统的核心指南，对于LangChain的企业级应用和长期维护具有重要意义。

**学习要求**：
- 掌握异步编程的最佳实践和设计模式
- 学会异步系统的架构设计和部署策略
- 理解异步代码的测试和调试方法
- 能够构建生产级的异步应用系统

**案例要求**：
- 实现完整的异步最佳实践指南
- 进行异步系统架构设计实践
- 应用测试和调试方法解决实际问题
- 验证点：能独立构建生产级异步系统

In [ ]:
print("🎯 异步最佳实践:")
print("=" * 50)

# 导入必要的库
import asyncio
import time
import datetime
import json
import random
import uuid
import logging
import signal
import sys
import os
import weakref
import inspect
import functools
import traceback
from typing import List, Dict, Any, Optional, Callable, Union, Type
from dataclasses import dataclass, asdict, field
from enum import Enum
from collections import defaultdict, deque
from contextlib import asynccontextmanager, contextmanager
import unittest.mock as mock

# 测试相关
import pytest
from unittest import TestCase, IsolatedAsyncioTestCase

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 异步编程最佳实践
print(f"📝 1. 异步编程最佳实践:")

# 1.1 异步上下文管理器最佳实践
print(f"\n   🔍 1.1 异步上下文管理器最佳实践:")

class AsyncResourceManager:
    """异步资源管理器 - 最佳实践示例"""
    
    def __init__(self, resource_name: str):
        self.resource_name = resource_name
        self.resource = None
        self._created_at = None
        self._logger = logging.getLogger(self.__class__.__name__)
    
    async def __aenter__(self):
        """进入上下文 - 资源初始化"""
        try:
            # 记录创建时间
            self._created_at = time.time()
            
            # 模拟资源创建
            await self._create_resource()
            
            self._logger.info(f"资源 {self.resource_name} 创建成功")
            return self
            
        except Exception as e:
            self._logger.error(f"资源 {self.resource_name} 创建失败: {e}")
            # 确保异常时资源被正确清理
            await self._cleanup_resource()
            raise
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """退出上下文 - 资源清理"""
        try:
            # 记录资源使用时间
            if self._created_at:
                usage_time = time.time() - self._created_at
                self._logger.info(f"资源 {self.resource_name} 使用时间: {usage_time:.2f}s")
            
            # 清理资源
            await self._cleanup_resource()
            
            # 处理异常
            if exc_type is not None:
                self._logger.error(f"资源 {self.resource_name} 使用中发生异常: {exc_val}")
                return False  # 不抑制异常
            
            self._logger.info(f"资源 {self.resource_name} 清理完成")
            return True
            
        except Exception as e:
            self._logger.error(f"资源 {self.resource_name} 清理失败: {e}")
            return False
    
    async def _create_resource(self):
        """创建资源 - 模拟实现"""
        await asyncio.sleep(0.1)  # 模拟创建时间
        self.resource = {"id": str(uuid.uuid4()), "name": self.resource_name}
    
    async def _cleanup_resource(self):
        """清理资源 - 模拟实现"""
        if self.resource:
            await asyncio.sleep(0.05)  # 模拟清理时间
            self.resource = None

# 1.2 异步异常处理最佳实践
print(f"\n   ⚠️ 1.2 异步异常处理最佳实践:")

@dataclass
class AsyncErrorContext:
    """异步错误上下文"""
    error_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    error_type: str = ""
    error_message: str = ""
    timestamp: float = field(default_factory=time.time)
    traceback_info: str = ""
    context_data: Dict[str, Any] = field(default_factory=dict)

class AsyncErrorHandler:
    """异步异常处理器 - 最佳实践示例"""
    
    def __init__(self):
        self.error_history: List[AsyncErrorContext] = []
        self.error_callbacks: Dict[Type[Exception], List[Callable]] = defaultdict(list)
        self._logger = logging.getLogger(self.__class__.__name__)
    
    def register_error_callback(self, exception_type: Type[Exception], 
                                callback: Callable[[AsyncErrorContext], None]):
        """注册异常回调"""
        self.error_callbacks[exception_type].append(callback)
    
    async def handle_exception(self, exception: Exception, 
                                context: Dict[str, Any] = None) -> AsyncErrorContext:
        """处理异常"""
        # 创建错误上下文
        error_context = AsyncErrorContext(
            error_type=type(exception).__name__,
            error_message=str(exception),
            traceback_info=traceback.format_exc(),
            context_data=context or {}
        )
        
        # 记录错误
        self.error_history.append(error_context)
        
        # 记录日志
        self._logger.error(
            f"异步异常 [{error_context.error_id}]: {error_context.error_type} - {error_context.error_message}"
        )
        
        # 执行回调
        await self._execute_error_callbacks(exception, error_context)
        
        return error_context
    
    async def _execute_error_callbacks(self, exception: Exception, 
                                       error_context: AsyncErrorContext):
        """执行异常回调"""
        exception_type = type(exception)
        
        # 查找匹配的回调
        callbacks = []
        for exc_type, cb_list in self.error_callbacks.items():
            if issubclass(exception_type, exc_type):
                callbacks.extend(cb_list)
        
        # 并发执行回调
        if callbacks:
            tasks = []
            for callback in callbacks:
                if asyncio.iscoroutinefunction(callback):
                    tasks.append(callback(error_context))
                else:
                    # 同步回调在线程池中执行
                    loop = asyncio.get_event_loop()
                    tasks.append(loop.run_in_executor(None, callback, error_context))
            
            await asyncio.gather(*tasks, return_exceptions=True)
    
    def get_error_statistics(self) -> Dict[str, Any]:
        """获取错误统计"""
        if not self.error_history:
            return {"total_errors": 0}
        
        error_types = defaultdict(int)
        recent_errors = [e for e in self.error_history 
                         if time.time() - e.timestamp < 3600]  # 最近1小时
        
        for error in self.error_history:
            error_types[error.error_type] += 1
        
        return {
            "total_errors": len(self.error_history),
            "recent_errors_1h": len(recent_errors),
            "error_types": dict(error_types),
            "most_common_error": max(error_types.items(), key=lambda x: x[1])[0] if error_types else None
        }

# 1.3 异步装饰器最佳实践
print(f"\n   🎨 1.3 异步装饰器最佳实践:")

def async_retry(max_attempts: int = 3, delay: float = 1.0, 
                 backoff: float = 2.0, exceptions: tuple = (Exception,)):
    """异步重试装饰器 - 最佳实践示例"""
    def decorator(func):
        @functools.wraps(func)
        async def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(max_attempts):
                try:
                    return await func(*args, **kwargs)
                    
                except exceptions as e:
                    last_exception = e
                    
                    if attempt == max_attempts - 1:
                        # 最后一次尝试失败
                        logging.error(f"函数 {func.__name__} 重试 {max_attempts} 次后仍然失败: {e}")
                        raise
                    
                    # 计算延迟时间
                    wait_time = delay * (backoff ** attempt)
                    logging.warning(f"函数 {func.__name__} 第 {attempt + 1} 次尝试失败: {e}, {wait_time:.1f}s 后重试")
                    await asyncio.sleep(wait_time)
            
            # 理论上不会到达这里
            raise last_exception
        
        return wrapper
    return decorator

def async_timeout(timeout_seconds: float):
    """异步超时装饰器 - 最佳实践示例"""
    def decorator(func):
        @functools.wraps(func)
        async def wrapper(*args, **kwargs):
            try:
                return await asyncio.wait_for(func(*args, **kwargs), timeout=timeout_seconds)
            except asyncio.TimeoutError:
                logging.error(f"函数 {func.__name__} 执行超时 ({timeout_seconds}s)")
                raise TimeoutError(f"函数 {func.__name__} 执行超时")
        
        return wrapper
    return decorator

def async_circuit_breaker(failure_threshold: int = 5, 
                          recovery_timeout: float = 60.0,
                          expected_exception: Type[Exception] = Exception):
    """异步断路器装饰器 - 最佳实践示例"""
    def decorator(func):
        # 断路器状态
        state = {
            "failure_count": 0,
            "last_failure_time": None,
            "state": "CLOSED"  # CLOSED, OPEN, HALF_OPEN
        }
        
        @functools.wraps(func)
        async def wrapper(*args, **kwargs):
            current_time = time.time()
            
            # 检查断路器状态
            if state["state"] == "OPEN":
                if current_time - state["last_failure_time"] > recovery_timeout:
                    state["state"] = "HALF_OPEN"
                    logging.info(f"断路器 {func.__name__} 进入半开状态")
                else:
                    raise Exception(f"断路器 {func.__name__} 处于开启状态")
            
            try:
                result = await func(*args, **kwargs)
                
                # 成功执行，重置失败计数
                if state["state"] == "HALF_OPEN":
                    state["state"] = "CLOSED"
                    logging.info(f"断路器 {func.__name__} 恢复到关闭状态")
                
                state["failure_count"] = 0
                return result
                
            except expected_exception as e:
                state["failure_count"] += 1
                state["last_failure_time"] = current_time
                
                if state["failure_count"] >= failure_threshold:
                    state["state"] = "OPEN"
                    logging.error(f"断路器 {func.__name__} 开启，失败次数: {state['failure_count']}")
                
                raise
        
        return wrapper
    return decorator

print(f"   ✅ 异步编程最佳实践完成")
print(f"      - AsyncResourceManager: 异步资源管理")
print(f"      - AsyncErrorHandler: 异步异常处理")
print(f"      - 异步装饰器: 重试、超时、断路器")

print(f"\n✅ 异步编程最佳实践完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握异步编程的最佳实践和设计模式")
print(f"   ✓ 学会异步系统的架构设计和部署策略")
print(f"   ✓ 理解异步代码的测试和调试方法")
print(f"   ✓ 能够构建生产级的异步应用系统")

### 异步系统架构设计 [⭐⭐进阶]
**知识点说明**：异步系统架构设计是构建可扩展、可维护异步应用的关键，对于LangChain的大规模部署和长期演进具有重要意义。

**学习要求**：
- 掌握异步系统的架构设计原则
- 学会微服务架构的异步通信
- 理解事件驱动架构的实现
- 能够设计高可用的异步系统

**案例要求**：
- 实现完整的异步系统架构
- 进行微服务异步通信实践
- 应用事件驱动架构解决实际问题
- 验证点：能独立设计高可用异步系统

In [ ]:
print("\n🏗️ 异步系统架构设计:")
print("=" * 50)

# 2. 异步系统架构设计
print(f"📝 2. 异步系统架构设计:")

# 2.1 微服务异步通信架构
print(f"\n   🔍 2.1 微服务异步通信架构:")

@dataclass
class ServiceMessage:
    """服务间消息"""
    message_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    source_service: str = ""
    target_service: str = ""
    message_type: str = ""
    payload: Dict[str, Any] = field(default_factory=dict)
    timestamp: float = field(default_factory=time.time)
    correlation_id: str = ""
    reply_to: str = ""

class AsyncServiceBus:
    """异步服务总线 - 微服务通信架构"""
    
    def __init__(self):
        # 消息队列
        self.message_queues: Dict[str, asyncio.Queue] = defaultdict(asyncio.Queue)
        self.subscriptions: Dict[str, List[Callable]] = defaultdict(list)
        
        # 服务注册
        self.services: Dict[str, Dict[str, Any]] = {}
        
        # 消息路由
        self.message_router: Dict[str, str] = {}
        
        # 统计信息
        self.stats = {
            "messages_sent": 0,
            "messages_delivered": 0,
            "messages_failed": 0,
            "active_services": 0
        }
        
        # 运行状态
        self.running = False
        self.dispatcher_task: Optional[asyncio.Task] = None
    
    async def start(self):
        """启动服务总线"""
        if self.running:
            return
        
        self.running = True
        self.dispatcher_task = asyncio.create_task(self._message_dispatcher())
        
        print(f"      🚀 异步服务总线启动")
    
    async def stop(self):
        """停止服务总线"""
        if not self.running:
            return
        
        self.running = False
        
        if self.dispatcher_task:
            self.dispatcher_task.cancel()
            try:
                await self.dispatcher_task
            except asyncio.CancelledError:
                pass
        
        print(f"      🛑 异步服务总线停止")
    
    def register_service(self, service_name: str, 
                         message_handler: Callable,
                         metadata: Dict[str, Any] = None):
        """注册服务"""
        self.services[service_name] = {
            "handler": message_handler,
            "metadata": metadata or {},
            "registered_at": time.time()
        }
        
        # 订阅服务队列
        self.subscribe_to_queue(service_name, message_handler)
        
        self.stats["active_services"] += 1
        print(f"      📝 服务注册: {service_name}")
    
    def subscribe_to_queue(self, queue_name: str, handler: Callable):
        """订阅消息队列"""
        self.subscriptions[queue_name].append(handler)
    
    async def send_message(self, message: ServiceMessage) -> str:
        """发送消息"""
        try:
            # 验证目标服务
            if message.target_service not in self.services:
                raise ValueError(f"目标服务 {message.target_service} 未注册")
            
            # 添加到目标队列
            await self.message_queues[message.target_service].put(message)
            
            self.stats["messages_sent"] += 1
            print(f"      📤 消息发送: {message.source_service} -> {message.target_service}")
            
            return message.message_id
            
        except Exception as e:
            self.stats["messages_failed"] += 1
            print(f"      ❌ 消息发送失败: {e}")
            raise
    
    async def broadcast_message(self, message: ServiceMessage, 
                                 exclude_services: List[str] = None):
        """广播消息"""
        exclude_services = exclude_services or []
        exclude_services.append(message.source_service)  # 排除发送者
        
        tasks = []
        for service_name in self.services:
            if service_name not in exclude_services:
                broadcast_msg = ServiceMessage(
                    source_service=message.source_service,
                    target_service=service_name,
                    message_type=message.message_type,
                    payload=message.payload,
                    correlation_id=message.correlation_id
                )
                tasks.append(self.send_message(broadcast_msg))
        
        # 并发发送广播消息
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        successful = sum(1 for r in results if not isinstance(r, Exception))
        print(f"      📢 广播消息完成: {successful}/{len(tasks)} 个服务")
        
        return successful
    
    async def _message_dispatcher(self):
        """消息分发器"""
        while self.running:
            try:
                # 并发处理所有队列
                tasks = []
                for queue_name, queue in self.message_queues.items():
                    if not queue.empty():
                        task = asyncio.create_task(
                            self._process_queue_message(queue_name)
                        )
                        tasks.append(task)
                
                if tasks:
                    await asyncio.gather(*tasks, return_exceptions=True)
                
                await asyncio.sleep(0.01)  # 短暂休眠避免CPU占用过高
                
            except Exception as e:
                print(f"      ❌ 消息分发器错误: {e}")
                await asyncio.sleep(0.1)
    
    async def _process_queue_message(self, queue_name: str):
        """处理队列消息"""
        try:
            message = await self.message_queues[queue_name].get()
            
            # 获取订阅者
            handlers = self.subscriptions.get(queue_name, [])
            
            # 并发处理消息
            tasks = []
            for handler in handlers:
                if asyncio.iscoroutinefunction(handler):
                    tasks.append(handler(message))
                else:
                    loop = asyncio.get_event_loop()
                    tasks.append(loop.run_in_executor(None, handler, message))
            
            results = await asyncio.gather(*tasks, return_exceptions=True)
            
            # 统计成功投递
            successful = sum(1 for r in results if not isinstance(r, Exception))
            if successful > 0:
                self.stats["messages_delivered"] += 1
            else:
                self.stats["messages_failed"] += 1
            
        except Exception as e:
            print(f"      ❌ 处理队列消息错误: {e}")
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        stats = self.stats.copy()
        stats["registered_services"] = list(self.services.keys())
        stats["queue_sizes"] = {
            name: queue.qsize() for name, queue in self.message_queues.items()
        }
        return stats

# 2.2 事件驱动架构
print(f"\n   🎯 2.2 事件驱动架构:")

@dataclass
class DomainEvent:
    """领域事件"""
    event_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    event_type: str = ""
    aggregate_id: str = ""
    event_data: Dict[str, Any] = field(default_factory=dict)
    timestamp: float = field(default_factory=time.time)
    version: int = 1

class AsyncEventBus:
    """异步事件总线 - 事件驱动架构"""
    
    def __init__(self):
        # 事件存储
        self.event_store: List[DomainEvent] = []
        
        # 事件处理器
        self.event_handlers: Dict[str, List[Callable]] = defaultdict(list)
        
        # 事件投影
        self.projections: Dict[str, Any] = {}
        
        # 事件流
        self.event_streams: Dict[str, asyncio.Queue] = defaultdict(asyncio.Queue)
        
        # 统计信息
        self.stats = {
            "events_published": 0,
            "events_processed": 0,
            "handlers_registered": 0,
            "projections_updated": 0
        }
        
        # 运行状态
        self.running = False
        self.processor_task: Optional[asyncio.Task] = None
    
    async def start(self):
        """启动事件总线"""
        if self.running:
            return
        
        self.running = True
        self.processor_task = asyncio.create_task(self._event_processor())
        
        print(f"      🚀 异步事件总线启动")
    
    async def stop(self):
        """停止事件总线"""
        if not self.running:
            return
        
        self.running = False
        
        if self.processor_task:
            self.processor_task.cancel()
            try:
                await self.processor_task
            except asyncio.CancelledError:
                pass
        
        print(f"      🛑 异步事件总线停止")
    
    def register_event_handler(self, event_type: str, handler: Callable):
        """注册事件处理器"""
        self.event_handlers[event_type].append(handler)
        self.stats["handlers_registered"] += 1
        print(f"      📝 事件处理器注册: {event_type}")
    
    async def publish_event(self, event: DomainEvent):
        """发布事件"""
        try:
            # 存储事件
            self.event_store.append(event)
            
            # 添加到事件流
            await self.event_streams[event.event_type].put(event)
            
            self.stats["events_published"] += 1
            print(f"      📤 事件发布: {event.event_type} (ID: {event.event_id[:8]})")
            
            return event.event_id
            
        except Exception as e:
            print(f"      ❌ 事件发布失败: {e}")
            raise
    
    async def create_projection(self, projection_name: str, 
                                initial_data: Any = None):
        """创建投影"""
        self.projections[projection_name] = initial_data or {}
        print(f"      📊 投影创建: {projection_name}")
    
    async def update_projection(self, projection_name: str, 
                                update_func: Callable[[Any, DomainEvent], Any]):
        """更新投影"""
        if projection_name not in self.projections:
            raise ValueError(f"投影 {projection_name} 不存在")
        
        # 注册投影更新处理器
        async def projection_updater(event: DomainEvent):
            try:
                current_data = self.projections[projection_name]
                new_data = update_func(current_data, event)
                self.projections[projection_name] = new_data
                self.stats["projections_updated"] += 1
            except Exception as e:
                print(f"      ❌ 投影更新失败: {e}")
        
        # 为所有事件类型注册投影更新器
        for event_type in self.event_handlers:
            self.event_handlers[event_type].append(projection_updater)
    
    async def get_events_by_aggregate(self, aggregate_id: str) -> List[DomainEvent]:
        """获取聚合的事件"""
        return [event for event in self.event_store 
                if event.aggregate_id == aggregate_id]
    
    async def replay_events(self, from_timestamp: float = 0):
        """重放事件"""
        events_to_replay = [event for event in self.event_store 
                           if event.timestamp >= from_timestamp]
        
        print(f"      🔄 开始重放事件: {len(events_to_replay)} 个事件")
        
        for event in events_to_replay:
            await self.event_streams[event.event_type].put(event)
        
        print(f"      ✅ 事件重放完成")
    
    async def _event_processor(self):
        """事件处理器"""
        while self.running:
            try:
                # 并发处理所有事件流
                tasks = []
                for event_type, stream in self.event_streams.items():
                    if not stream.empty():
                        task = asyncio.create_task(
                            self._process_event_stream(event_type)
                        )
                        tasks.append(task)
                
                if tasks:
                    await asyncio.gather(*tasks, return_exceptions=True)
                
                await asyncio.sleep(0.01)
                
            except Exception as e:
                print(f"      ❌ 事件处理器错误: {e}")
                await asyncio.sleep(0.1)
    
    async def _process_event_stream(self, event_type: str):
        """处理事件流"""
        try:
            event = await self.event_streams[event_type].get()
            
            # 获取事件处理器
            handlers = self.event_handlers.get(event_type, [])
            
            # 并发处理事件
            tasks = []
            for handler in handlers:
                if asyncio.iscoroutinefunction(handler):
                    tasks.append(handler(event))
                else:
                    loop = asyncio.get_event_loop()
                    tasks.append(loop.run_in_executor(None, handler, event))
            
            results = await asyncio.gather(*tasks, return_exceptions=True)
            
            # 统计成功处理
            successful = sum(1 for r in results if not isinstance(r, Exception))
            if successful > 0:
                self.stats["events_processed"] += 1
            
        except Exception as e:
            print(f"      ❌ 处理事件流错误: {e}")
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        stats = self.stats.copy()
        stats["total_events"] = len(self.event_store)
        stats["event_types"] = list(self.event_handlers.keys())
        stats["projections"] = list(self.projections.keys())
        stats["stream_sizes"] = {
            event_type: stream.qsize() for event_type, stream in self.event_streams.items()
        }
        return stats

print(f"   ✅ 异步系统架构设计完成")
print(f"      - AsyncServiceBus: 微服务异步通信")
print(f"      - AsyncEventBus: 事件驱动架构")
print(f"      - 支持高可用的分布式异步系统")

print(f"\n✅ 异步系统架构设计完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握异步系统的架构设计原则")
print(f"   ✓ 学会微服务架构的异步通信")
print(f"   ✓ 理解事件驱动架构的实现")
print(f"   ✓ 能够设计高可用的异步系统")

### LangChain异步最佳实践 [⭐⭐进阶]
**知识点说明**：LangChain异步最佳实践是掌握AI系统异步开发的核心，对于LangChain的高性能、高可用部署具有重要意义。

**学习要求**：
- 掌握LangChain异步开发的最佳实践
- 学会AI模型的异步调用和管理
- 理解LangChain工作流的异步优化
- 能够构建企业级LangChain异步应用

**案例要求**：
- 实现完整的LangChain异步最佳实践
- 进行AI模型异步调用实践
- 应用工作流异步优化解决实际问题
- 验证点：能独立构建企业级LangChain异步应用

In [ ]:
print("\n🤖 LangChain异步最佳实践:")
print("=" * 50)

# 3. LangChain异步最佳实践
print(f"📝 3. LangChain异步最佳实践:")

# 3.1 AI模型异步调用管理
print(f"\n   🔍 3.1 AI模型异步调用管理:")

@dataclass
class ModelConfig:
    """模型配置"""
    model_name: str
    max_tokens: int = 1000
    temperature: float = 0.7
    timeout: float = 30.0
    retry_attempts: int = 3
    rate_limit: int = 10  # 每秒请求数

@dataclass
class ModelRequest:
    """模型请求"""
    request_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    prompt: str = ""
    config: ModelConfig = None
    context: Dict[str, Any] = field(default_factory=dict)
    priority: int = 1
    created_at: float = field(default_factory=time.time)

@dataclass
class ModelResponse:
    """模型响应"""
    request_id: str
    response: str
    tokens_used: int
    processing_time: float
    model_name: str
    success: bool
    error_message: str = ""

class AsyncModelManager:
    """异步模型管理器 - LangChain最佳实践"""
    
    def __init__(self):
        # 模型配置
        self.model_configs: Dict[str, ModelConfig] = {}
        
        # 请求队列
        self.request_queues: Dict[str, asyncio.PriorityQueue] = defaultdict(asyncio.PriorityQueue)
        
        # 速率限制
        self.rate_limiters: Dict[str, asyncio.Semaphore] = {}
        
        # 响应Future
        self.response_futures: Dict[str, asyncio.Future] = {}
        
        # 模型工作器
        self.model_workers: Dict[str, asyncio.Task] = {}
        
        # 统计信息
        self.stats = {
            "total_requests": 0,
            "successful_requests": 0,
            "failed_requests": 0,
            "average_response_time": 0.0,
            "total_tokens_used": 0
        }
        
        # 运行状态
        self.running = False
    
    async def start(self):
        """启动模型管理器"""
        if self.running:
            return
        
        self.running = True
        
        # 启动所有模型的工作器
        for model_name in self.model_configs:
            await self._start_model_worker(model_name)
        
        print(f"      🚀 异步模型管理器启动")
    
    async def stop(self):
        """停止模型管理器"""
        if not self.running:
            return
        
        self.running = False
        
        # 停止所有工作器
        for worker_task in self.model_workers.values():
            worker_task.cancel()
            try:
                await worker_task
            except asyncio.CancelledError:
                pass
        
        print(f"      🛑 异步模型管理器停止")
    
    def register_model(self, config: ModelConfig):
        """注册模型"""
        self.model_configs[config.model_name] = config
        self.rate_limiters[config.model_name] = asyncio.Semaphore(config.rate_limit)
        
        if self.running:
            asyncio.create_task(self._start_model_worker(config.model_name))
        
        print(f"      📝 模型注册: {config.model_name}")
    
    async def submit_request(self, request: ModelRequest) -> str:
        """提交模型请求"""
        if not request.config or request.config.model_name not in self.model_configs:
            raise ValueError(f"模型 {request.config.model_name if request.config else 'Unknown'} 未注册")
        
        # 创建响应Future
        response_future = asyncio.Future()
        self.response_futures[request.request_id] = response_future
        
        # 添加到请求队列
        await self.request_queues[request.config.model_name].put(
            (request.priority, request)
        )
        
        self.stats["total_requests"] += 1
        print(f"      📤 请求提交: {request.config.model_name} (ID: {request.request_id[:8]})")
        
        return request.request_id
    
    async def get_response(self, request_id: str, timeout: float = None) -> Optional[ModelResponse]:
        """获取模型响应"""
        if request_id not in self.response_futures:
            return None
        
        try:
            if timeout:
                response = await asyncio.wait_for(
                    self.response_futures[request_id], timeout=timeout
                )
            else:
                response = await self.response_futures[request_id]
            
            return response
            
        except asyncio.TimeoutError:
            return None
        finally:
            self.response_futures.pop(request_id, None)
    
    async def _start_model_worker(self, model_name: str):
        """启动模型工作器"""
        if model_name in self.model_workers:
            return
        
        worker_task = asyncio.create_task(self._model_worker_loop(model_name))
        self.model_workers[model_name] = worker_task
    
    async def _model_worker_loop(self, model_name: str):
        """模型工作器循环"""
        config = self.model_configs[model_name]
        rate_limiter = self.rate_limiters[model_name]
        
        while self.running:
            try:
                # 获取请求
                priority, request = await self.request_queues[model_name].get()
                
                # 速率限制
                async with rate_limiter:
                    # 处理请求
                    response = await self._process_model_request(request, config)
                    
                    # 设置响应
                    if request.request_id in self.response_futures:
                        self.response_futures[request.request_id].set_result(response)
                    
                    # 更新统计
                    self._update_stats(response)
            
            except Exception as e:
                print(f"      ❌ 模型工作器错误 ({model_name}): {e}")
                await asyncio.sleep(1)
    
    async def _process_model_request(self, request: ModelRequest, 
                                    config: ModelConfig) -> ModelResponse:
        """处理模型请求"""
        start_time = time.time()
        
        try:
            # 模拟AI模型调用
            await asyncio.sleep(random.uniform(0.5, 2.0))  # 模拟处理时间
            
            # 模拟响应生成
            response_text = f"AI模型 {config.model_name} 对 '{request.prompt[:50]}...' 的响应"
            tokens_used = random.randint(100, min(config.max_tokens, len(response_text)))
            
            processing_time = time.time() - start_time
            
            return ModelResponse(
                request_id=request.request_id,
                response=response_text,
                tokens_used=tokens_used,
                processing_time=processing_time,
                model_name=config.model_name,
                success=True
            )
            
        except Exception as e:
            processing_time = time.time() - start_time
            
            return ModelResponse(
                request_id=request.request_id,
                response="",
                tokens_used=0,
                processing_time=processing_time,
                model_name=config.model_name,
                success=False,
                error_message=str(e)
            )
    
    def _update_stats(self, response: ModelResponse):
        """更新统计信息"""
        if response.success:
            self.stats["successful_requests"] += 1
            self.stats["total_tokens_used"] += response.tokens_used
        else:
            self.stats["failed_requests"] += 1
        
        # 更新平均响应时间
        total_successful = self.stats["successful_requests"]
        if total_successful > 0:
            current_avg = self.stats["average_response_time"]
            self.stats["average_response_time"] = (
                (current_avg * (total_successful - 1) + response.processing_time) / total_successful
            )
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        stats = self.stats.copy()
        stats["registered_models"] = list(self.model_configs.keys())
        stats["queue_sizes"] = {
            model: queue.qsize() for model, queue in self.request_queues.items()
        }
        stats["pending_responses"] = len(self.response_futures)
        
        if stats["total_requests"] > 0:
            stats["success_rate"] = stats["successful_requests"] / stats["total_requests"]
        else:
            stats["success_rate"] = 0
        
        return stats

# 3.2 LangChain工作流异步优化
print(f"\n   🔄 3.2 LangChain工作流异步优化:")

@dataclass
class WorkflowStep:
    """工作流步骤"""
    step_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    step_name: str = ""
    step_type: str = ""  # llm_call, data_processing, validation, etc.
    config: Dict[str, Any] = field(default_factory=dict)
    dependencies: List[str] = field(default_factory=list)
    retry_config: Dict[str, Any] = field(default_factory=dict)
    timeout: float = 30.0

@dataclass
class WorkflowExecution:
    """工作流执行"""
    execution_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    workflow_id: str = ""
    steps: List[WorkflowStep] = field(default_factory=list)
    context: Dict[str, Any] = field(default_factory=dict)
    status: str = "pending"  # pending, running, completed, failed
    started_at: float = field(default_factory=time.time)
    completed_at: Optional[float] = None
    step_results: Dict[str, Any] = field(default_factory=dict)
    step_errors: Dict[str, str] = field(default_factory=dict)

class AsyncWorkflowEngine:
    """异步工作流引擎 - LangChain最佳实践"""
    
    def __init__(self):
        # 工作流定义
        self.workflows: Dict[str, List[WorkflowStep]] = {}
        
        # 执行管理
        self.executions: Dict[str, WorkflowExecution] = {}
        self.execution_queue: asyncio.Queue = asyncio.Queue()
        
        # 步骤处理器
        self.step_processors: Dict[str, Callable] = {}
        
        # 执行统计
        self.stats = {
            "total_executions": 0,
            "completed_executions": 0,
            "failed_executions": 0,
            "average_execution_time": 0.0,
            "parallel_executions": 0
        }
        
        # 运行状态
        self.running = False
        self.executor_task: Optional[asyncio.Task] = None
    
    async def start(self):
        """启动工作流引擎"""
        if self.running:
            return
        
        self.running = True
        self.executor_task = asyncio.create_task(self._execution_loop())
        
        print(f"      🚀 异步工作流引擎启动")
    
    async def stop(self):
        """停止工作流引擎"""
        if not self.running:
            return
        
        self.running = False
        
        if self.executor_task:
            self.executor_task.cancel()
            try:
                await self.executor_task
            except asyncio.CancelledError:
                pass
        
        print(f"      🛑 异步工作流引擎停止")
    
    def register_workflow(self, workflow_id: str, steps: List[WorkflowStep]):
        """注册工作流"""
        self.workflows[workflow_id] = steps
        print(f"      📝 工作流注册: {workflow_id} ({len(steps)} 个步骤)")
    
    def register_step_processor(self, step_type: str, processor: Callable):
        """注册步骤处理器"""
        self.step_processors[step_type] = processor
        print(f"      🔧 步骤处理器注册: {step_type}")
    
    async def execute_workflow(self, workflow_id: str, 
                                context: Dict[str, Any] = None) -> str:
        """执行工作流"""
        if workflow_id not in self.workflows:
            raise ValueError(f"工作流 {workflow_id} 未注册")
        
        # 创建执行实例
        execution = WorkflowExecution(
            workflow_id=workflow_id,
            steps=self.workflows[workflow_id].copy(),
            context=context or {}
        )
        
        self.executions[execution.execution_id] = execution
        await self.execution_queue.put(execution)
        
        self.stats["total_executions"] += 1
        print(f"      🚀 工作流执行: {workflow_id} (ID: {execution.execution_id[:8]})")
        
        return execution.execution_id
    
    async def get_execution_status(self, execution_id: str) -> Optional[WorkflowExecution]:
        """获取执行状态"""
        return self.executions.get(execution_id)
    
    async def _execution_loop(self):
        """执行循环"""
        while self.running:
            try:
                # 获取执行任务
                execution = await self.execution_queue.get()
                
                # 并发执行工作流
                asyncio.create_task(self._execute_workflow_async(execution))
                
            except Exception as e:
                print(f"      ❌ 执行循环错误: {e}")
                await asyncio.sleep(0.1)
    
    async def _execute_workflow_async(self, execution: WorkflowExecution):
        """异步执行工作流"""
        try:
            execution.status = "running"
            self.stats["parallel_executions"] += 1
            
            # 构建依赖图
            dependency_graph = self._build_dependency_graph(execution.steps)
            
            # 按依赖顺序执行步骤
            await self._execute_steps_with_dependencies(execution, dependency_graph)
            
            # 完成执行
            execution.status = "completed"
            execution.completed_at = time.time()
            
            self.stats["completed_executions"] += 1
            self._update_execution_stats(execution)
            
            print(f"      ✅ 工作流完成: {execution.workflow_id} (ID: {execution.execution_id[:8]})")
            
        except Exception as e:
            execution.status = "failed"
            execution.completed_at = time.time()
            
            self.stats["failed_executions"] += 1
            print(f"      ❌ 工作流失败: {execution.workflow_id} - {e}")
            
        finally:
            self.stats["parallel_executions"] -= 1
    
    def _build_dependency_graph(self, steps: List[WorkflowStep]) -> Dict[str, List[str]]:
        """构建依赖图"""
        graph = {}
        for step in steps:
            graph[step.step_id] = step.dependencies.copy()
        return graph
    
    async def _execute_steps_with_dependencies(self, execution: WorkflowExecution, 
                                              dependency_graph: Dict[str, List[str]]):
        """按依赖顺序执行步骤"""
        completed_steps = set()
        step_map = {step.step_id: step for step in execution.steps}
        
        while len(completed_steps) < len(execution.steps):
            # 找到可以执行的步骤（依赖已完成）
            ready_steps = []
            for step_id, dependencies in dependency_graph.items():
                if step_id not in completed_steps and all(dep in completed_steps for dep in dependencies):
                    ready_steps.append(step_id)
            
            if not ready_steps:
                raise Exception("检测到循环依赖或无法满足的依赖")
            
            # 并发执行准备好的步骤
            tasks = []
            for step_id in ready_steps:
                step = step_map[step_id]
                task = asyncio.create_task(
                    self._execute_step(execution, step)
                )
                tasks.append((step_id, task))
            
            # 等待所有步骤完成
            for step_id, task in tasks:
                try:
                    result = await task
                    execution.step_results[step_id] = result
                    completed_steps.add(step_id)
                except Exception as e:
                    execution.step_errors[step_id] = str(e)
                    raise
    
    async def _execute_step(self, execution: WorkflowExecution, 
                           step: WorkflowStep) -> Any:
        """执行单个步骤"""
        processor = self.step_processors.get(step.step_type)
        if not processor:
            raise ValueError(f"步骤类型 {step.step_type} 没有注册处理器")
        
        try:
            # 应用超时
            if asyncio.iscoroutinefunction(processor):
                result = await asyncio.wait_for(
                    processor(execution.context, step.config),
                    timeout=step.timeout
                )
            else:
                loop = asyncio.get_event_loop()
                result = await asyncio.wait_for(
                    loop.run_in_executor(None, processor, execution.context, step.config),
                    timeout=step.timeout
                )
            
            # 更新上下文
            execution.context[f"step_{step.step_id}_result"] = result
            
            return result
            
        except asyncio.TimeoutError:
            raise Exception(f"步骤 {step.step_name} 执行超时")
    
    def _update_execution_stats(self, execution: WorkflowExecution):
        """更新执行统计"""
        if execution.completed_at and execution.started_at:
            execution_time = execution.completed_at - execution.started_at
            
            total_completed = self.stats["completed_executions"]
            if total_completed > 0:
                current_avg = self.stats["average_execution_time"]
                self.stats["average_execution_time"] = (
                    (current_avg * (total_completed - 1) + execution_time) / total_completed
                )
    
    def get_stats(self) -> Dict[str, Any]:
        """获取统计信息"""
        stats = self.stats.copy()
        stats["registered_workflows"] = list(self.workflows.keys())
        stats["active_executions"] = self.stats["parallel_executions"]
        stats["queue_size"] = self.execution_queue.qsize()
        
        if stats["total_executions"] > 0:
            stats["success_rate"] = stats["completed_executions"] / stats["total_executions"]
        else:
            stats["success_rate"] = 0
        
        return stats

print(f"   ✅ LangChain异步最佳实践完成")
print(f"      - AsyncModelManager: AI模型异步调用管理")
print(f"      - AsyncWorkflowEngine: LangChain工作流异步优化")
print(f"      - 支持企业级LangChain异步应用开发")

print(f"\n✅ LangChain异步最佳实践完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握LangChain异步开发的最佳实践")
print(f"   ✓ 学会AI模型的异步调用和管理")
print(f"   ✓ 理解LangChain工作流的异步优化")
print(f"   ✓ 能够构建企业级LangChain异步应用")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**6.9 异步最佳实践 [⭐⭐进阶]**
- ✅ 掌握异步编程的最佳实践和设计模式
- ✅ 学会异步系统的架构设计和部署策略
- ✅ 理解异步代码的测试和调试方法
- ✅ 能够构建生产级的异步应用系统
- ✅ 掌握异步系统的架构设计原则
- ✅ 学会微服务架构的异步通信
- ✅ 理解事件驱动架构的实现
- ✅ 能够设计高可用的异步系统
- ✅ 掌握LangChain异步开发的最佳实践
- ✅ 学会AI模型的异步调用和管理
- ✅ 理解LangChain工作流的异步优化
- ✅ 能够构建企业级LangChain异步应用
- ✅ 能独立构建生产级异步系统
- ✅ 能独立设计高可用异步系统
- ✅ 能独立构建企业级LangChain异步应用

### 🎯 与LangChain学习的关联

**异步最佳实践重要性**：
- 最佳实践确保LangChain的稳定性
- 架构设计支持LangChain的扩展性
- 测试方法保障LangChain的质量
- 部署策略实现LangChain的可靠性
- 企业级应用满足LangChain的商业需求

**实际应用场景**：
- LangChain的大规模生产部署
- LangChain的微服务架构实现
- LangChain的事件驱动系统设计
- LangChain的企业级AI平台构建
- LangChain的高并发对话系统开发

### 📚 进阶学习建议

1. **实践建议**：
   - 深入实践异步系统架构设计
   - 掌握微服务异步通信的高级模式
   - 学习事件驱动架构的复杂应用

2. **扩展学习**：
   - 了解分布式异步系统设计
   - 学习云原生异步应用架构
   - 探索AI系统的异步优化策略

3. **实际应用**：
   - 构建生产级LangChain异步平台
   - 开发企业级异步监控系统
   - 实现智能化的异步运维系统

### 🔧 常见错误与注意事项

1. **资源管理不当**：
   ```python
   # 错误：没有正确管理异步资源
   async def bad_resource_management():
       connection = await create_connection()
       await connection.execute(query)
       # 忘记关闭连接，导致资源泄漏
   
   # 正确：使用异步上下文管理器
   async def good_resource_management():
       async with AsyncConnectionManager() as conn:
           await conn.execute(query)
       # 连接自动关闭
   ```

2. **异常处理缺失**：
   ```python
   # 错误：没有适当的异常处理
   async def unhandled_exceptions():
       result = await risky_operation()
       return process(result)  # 可能抛出未处理的异常
   
   # 正确：全面的异常处理
   async def proper_exception_handling():
       try:
           result = await risky_operation()
           return process(result)
       except SpecificException as e:
           logger.error(f"特定异常: {e}")
           await handle_specific_error(e)
       except Exception as e:
           logger.error(f"未知异常: {e}")
           await handle_general_error(e)
   ```

3. **同步阻塞异步**：
   ```python
   # 错误：在异步代码中使用同步阻塞操作
   async def blocking_async_code():
       data = sync_heavy_computation()  # 阻塞事件循环
       return await process_async(data)
   
   # 正确：使用异步替代方案或线程池
   async def non_blocking_async_code():
       loop = asyncio.get_event_loop()
       data = await loop.run_in_executor(None, sync_heavy_computation)
       return await process_async(data)
   ```

4. **过度并发问题**：
   ```python
   # 错误：无限制并发导致资源耗尽
   async def unlimited_concurrency():
       tasks = [process_item(item) for item in large_list]
       results = await asyncio.gather(*tasks)  # 可能创建过多任务
   
   # 正确：使用信号量控制并发数
   async def controlled_concurrency():
       semaphore = asyncio.Semaphore(100)  # 限制并发数
       
       async def process_with_semaphore(item):
           async with semaphore:
               return await process_item(item)
       
       tasks = [process_with_semaphore(item) for item in large_list]
       results = await asyncio.gather(*tasks)
   ```

5. **缺乏监控和日志**：
   ```python
   # 错误：没有监控和日志记录
   async def unmonitored_service():
       while True:
           await process_requests()  # 不知道性能和错误情况
   
   # 正确：完整的监控和日志系统
   async def monitored_service():
       monitor = AsyncPerformanceMonitor()
       await monitor.start_monitoring()
       
       try:
           while True:
               start_time = time.time()
               
               try:
                   await process_requests()
                   logger.info("请求处理成功")
               except Exception as e:
                   logger.error(f"请求处理失败: {e}")
                   await handle_error(e)
               
               # 记录性能指标
               processing_time = time.time() - start_time
               monitor.record_processing_time(processing_time)
               
               # 检查性能告警
               if monitor.get_average_latency() > threshold:
                   await trigger_performance_alert()
       
       finally:
           await monitor.stop_monitoring()
   ```

6. **测试覆盖不足**：
   ```python
   # 错误：没有异步测试
   def test_sync_only():
       result = sync_function()
       assert result == expected
   
   # 正确：全面的异步测试
   class TestAsyncFunction(IsolatedAsyncioTestCase):
       async def test_async_function_success(self):
           result = await async_function()
           self.assertEqual(result, expected)
       
       async def test_async_function_exception(self):
           with self.assertRaises(ExpectedException):
               await async_function_with_error()
       
       async def test_async_function_timeout(self):
           with self.assertRaises(asyncio.TimeoutError):
               await asyncio.wait_for(slow_async_function(), timeout=1.0)
   ```

### 🌐 异步最佳实践总结

**架构设计原则**：
- 优先使用异步上下文管理器管理资源
- 实现全面的异常处理和错误恢复
- 采用事件驱动架构提高系统响应性
- 使用微服务异步通信增强系统扩展性

**性能优化策略**：
- 合理控制并发数量避免资源耗尽
- 使用批处理和缓存提高处理效率
- 实现智能的负载均衡和故障转移
- 建立完善的监控和告警机制

**LangChain特定实践**：
- 实现AI模型的异步调用管理
- 构建可扩展的工作流执行引擎
- 采用事件驱动架构处理AI交互
- 建立企业级的监控和运维体系

---

**🎉 恭喜完成异步编程全部学习！**

你已经全面掌握了异步编程的核心技术、性能优化方法和最佳实践，能够构建生产级的异步系统，为LangChain智能应用提供了强大的技术支撑。

## 🚀 学习成果总结

**第六节：异步编程 - 全部完成** ✅
- 6.1 异步编程基础 ✅
- 6.2 协程与事件循环 ✅
- 6.3 异步I/O操作 ✅
- 6.4 异步网络编程 ✅
- 6.5 异步Web框架 ✅
- 6.6 并发编程模式 ✅
- 6.7 异步任务队列 ✅
- 6.8 异步性能优化 ✅
- 6.9 异步最佳实践 ✅

**核心能力掌握**：
- ✅ 异步编程基础理论和实践
- ✅ 协程和事件循环的深入理解
- ✅ 异步I/O和网络编程技术
- ✅ 异步Web框架和应用开发
- ✅ 并发编程模式和任务队列
- ✅ 异步性能分析和优化
- ✅ 异步系统最佳实践和架构设计

## 🎯 下一阶段学习预告

**第七节：Web开发技术** 🌐
- 7.1 Web框架基础
- 7.2 HTTP协议与REST API
- 7.3 数据库交互与ORM
- 7.4 前后端分离架构
- 7.5 Web安全与认证
- 7.6 缓存策略与优化
- 7.7 微服务架构
- 7.8 容器化部署
- 7.9 Web性能优化

**后续章节预告**：
- 第八节：项目工程实践

继续加油，异步编程已经完全掌握！准备进入Web开发技术的学习！